<a href="https://colab.research.google.com/github/yingchenjin/IANAL/blob/main/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
import time

"""
Script used to find entries of Stack Exchange sites that contain the acronym "IANAL" and
save them to a CSV file. The script will search for questions and answers containing "IANAL"
before saving the data.
"""

# Configuration
SITE = 'softwareengineering'  # Change to another Stack Exchange site as needed
OUTPUT_FILE = 'ianal_posts.csv'
API_KEY = 'rl_mQJm8BgRoQ5o9oj8igSfNR66y'  # Add your API key here for higher rate limits (optional)

# API endpoints
QUESTION_SEARCH_URL = "https://api.stackexchange.com/2.3/search/advanced"
QUESTION_ANSWERS_URL = "https://api.stackexchange.com/2.3/questions/{id}/answers"
ANSWER_DETAIL_URL = "https://api.stackexchange.com/2.3/answers/{id}"
QUESTION_DETAIL_URL = "https://api.stackexchange.com/2.3/questions/{id}"
EXCERPT_SEARCH_URL = "https://api.stackexchange.com/2.3/search/excerpts"

# Track processed answer IDs to avoid duplicates
processed_answer_ids = set()

# Function to safely make API requests with error handling
def make_request(url, params):
    """Helper function to make API requests with retries and error handling."""
    for _ in range(3):  # Retry up to 3 times in case of failure
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded. Waiting before retrying...")
                time.sleep(5)  # Wait longer for rate limits
            else:
                print(f"Error: Received status code {response.status_code}")
                print("Response Text:", response.text[:500])
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(2)  # Short wait before retrying
    return None

# Open the CSV file for writing
with open(OUTPUT_FILE, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # CSV header row
    writer.writerow(["post_type", "post_id", "creation_date", "question_id", "question_title", "question_body", "answer_body"])

    ###########################################################################
    # PART 1: Process questions that contain "IANAL" using /search/advanced
    ###########################################################################
    print("Processing questions that contain IANAL...")
    page = 1
    while True:
        params = {
            'q': 'IANAL',
            'site': SITE,
            'pagesize': 100,
            'page': page,
            'filter': 'withbody',
            'key': API_KEY  # Use API key if available
        }
        data = make_request(QUESTION_SEARCH_URL, params)
        if not data or 'items' not in data:
            print("No more question items or an error occurred.")
            break

        items = data.get('items', [])
        print(f"Page {page} | Returned Items: {len(items)} | Has More: {data.get('has_more')}")

        for question in items:
            if "ianal" in question.get('body', '').lower():
                question_id = question.get('question_id')
                creation_date = question.get('creation_date')
                title = question.get('title')
                question_body = question.get('body')

                # Write question row
                writer.writerow(["question", question_id, creation_date, question_id, title, question_body, ""])

                # Fetch answers
                ans_params = {'site': SITE, 'pagesize': 100, 'filter': 'withbody', 'key': API_KEY}
                ans_data = make_request(QUESTION_ANSWERS_URL.format(id=question_id), ans_params)

                # Process answers
                if ans_data and 'items' in ans_data:
                    for answer in ans_data['items']:
                        answer_body = answer.get('body', '')
                        if "ianal" in answer_body.lower():
                            answer_id = answer.get('answer_id')
                            creation_date_ans = answer.get('creation_date')
                            if answer_id not in processed_answer_ids:
                                writer.writerow(["answer", answer_id, creation_date_ans, question_id, title, question_body, answer_body])
                                processed_answer_ids.add(answer_id)

                time.sleep(0.2)  # Pause to respect API limits

        if not data.get('has_more'):
            break

        page += 1
        time.sleep(1)  # Slight delay to avoid hitting rate limits

    ##################################################################################
    # PART 2: Process answers that contain IANAL (but whose parent question did not)
    ##################################################################################
    print("Processing answers that contain IANAL...")
    page = 1
    while True:
        params = {'q': 'IANAL', 'site': SITE, 'pagesize': 100, 'page': page, 'key': API_KEY}
        data = make_request(EXCERPT_SEARCH_URL, params)
        if not data or 'items' not in data:
            print("No more excerpt items or an error occurred.")
            break

        items = data.get('items', [])
        print(f"Page {page} | Returned Items: {len(items)} | Has More: {data.get('has_more')}")

        for item in items:
            if item.get('item_type') == 'answer':
                answer_id = item.get('answer_id')
                if answer_id in processed_answer_ids or answer_id is None:
                    continue

                # Fetch full answer details
                ans_params = {'site': SITE, 'filter': 'withbody', 'key': API_KEY}
                ans_data = make_request(ANSWER_DETAIL_URL.format(id=answer_id), ans_params)
                if not ans_data or 'items' not in ans_data:
                    continue

                answer_detail = ans_data['items'][0]
                answer_body = answer_detail.get('body', '')
                if "ianal" not in answer_body.lower():
                    continue

                creation_date_ans = answer_detail.get('creation_date')
                question_id = answer_detail.get('question_id')

                # Fetch parent question details
                quest_params = {'site': SITE, 'filter': 'withbody', 'key': API_KEY}
                quest_data = make_request(QUESTION_DETAIL_URL.format(id=question_id), quest_params)
                if not quest_data or 'items' not in quest_data:
                    continue

                question_detail = quest_data['items'][0]
                title = question_detail.get('title')
                question_body = question_detail.get('body')

                # Write row for answer and parent question
                writer.writerow(["answer", answer_id, creation_date_ans, question_id, title, question_body, answer_body])
                processed_answer_ids.add(answer_id)

                time.sleep(0.2)  # Pause to respect API limits

        if not data.get('has_more'):
            break

        page += 1
        time.sleep(1)  # Slight delay to avoid hitting rate limits

print(f"Data saved to {OUTPUT_FILE}")


Processing questions that contain IANAL...
Error: Received status code 403
Response Text: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetic
No more question items or an error occurred.
Processing answers that contain IANAL...
Error: Received status code 403
Response Text: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=d

In [ ]:
import requests
import csv
import time

"""
Script used to find entries of Stack Exchange sites that contain the acronym "IANAL" and
retrieve all answers for those questions, along with upvotes and number of answers per question.
"""

# Configuration
SITE = 'softwareengineering'  # Change to another Stack Exchange site as needed
OUTPUT_FILE = 'ianal_posts.csv'
API_KEY = 'rl_gkMMYsnHwp1tyui3hWdjFRN5U'  # Add your API key here for higher rate limits (optional)

# API endpoints
ANSWER_SEARCH_URL = "https://api.stackexchange.com/2.3/search/excerpts"
QUESTION_ANSWERS_URL = "https://api.stackexchange.com/2.3/questions/{id}/answers"
QUESTION_DETAIL_URL = "https://api.stackexchange.com/2.3/questions/{id}"

def make_request(url, params):
    """Helper function to make API requests with retries and error handling."""
    for _ in range(3):  # Retry up to 3 times in case of failure
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded. Waiting before retrying...")
                time.sleep(5)
            else:
                print(f"Error: Received status code {response.status_code}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(2)
    return None

with open(OUTPUT_FILE, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["question_id", "question_title", "question_body", "num_answers", "answers_with_upvotes"])

    print("Searching for answers that explicitly contain 'IANAL'...")
    page = 1
    while True:
        params = {
            'q': 'IANAL',
            'site': SITE,
            'pagesize': 100,
            'page': page,
            'filter': 'withbody',
            'item_type': 'answer',
            'key': API_KEY
        }
        data = make_request(ANSWER_SEARCH_URL, params)
        if not data or 'items' not in data:
            break

        question_ids = set()
        for item in data.get('items', []):
            if item.get('item_type') == 'answer':
                question_ids.add(item.get('question_id'))

        for question_id in question_ids:
            # Fetch question details
            quest_params = {'site': SITE, 'filter': 'withbody', 'key': API_KEY}
            quest_data = make_request(QUESTION_DETAIL_URL.format(id=question_id), quest_params)

            # Fetch all answers for the question
            ans_params = {'site': SITE, 'pagesize': 100, 'filter': 'withbody', 'key': API_KEY}
            ans_data = make_request(QUESTION_ANSWERS_URL.format(id=question_id), ans_params)

            if not quest_data or 'items' not in quest_data or not ans_data or 'items' not in ans_data:
                continue

            question_detail = quest_data['items'][0]
            question_title = question_detail.get('title', '')
            question_body = question_detail.get('body', '')
            num_answers = len(ans_data['items'])

            # Store all answers and their upvotes in a structured format
            answers_with_upvotes = []
            for answer in ans_data['items']:
                answer_id = answer.get('answer_id')
                answer_body = answer.get('body', '').replace('\n', ' ').replace('\r', ' ')
                upvotes = answer.get('score', 0)
                answers_with_upvotes.append(f"[Answer ID: {answer_id}, Upvotes: {upvotes}] {answer_body}")

            writer.writerow([question_id, question_title, question_body, num_answers, " | ".join(answers_with_upvotes)])

            time.sleep(0.2)

        if not data.get('has_more'):
            break

        page += 1
        time.sleep(1)

print(f"Data saved to {OUTPUT_FILE}")


Searching for answers that explicitly contain 'IANAL'...
Error: Received status code 403
Data saved to ianal_posts.csv


In [ ]:
import requests
import csv
import time

"""
Script used to find entries of Stack Exchange sites that contain the acronym "IANAL" and
retrieve all answers for those questions, along with upvotes and number of answers per question.
"""

# Configuration
SITE = 'opensource'  # Change to another Stack Exchange site as needed
OUTPUT_FILE = 'uvoss.csv'
API_KEY = 'rl_HzTM85avzXjeFbbNQArcyrx67'  # Add your API key here for higher rate limits (optional)

# API endpoints
ANSWER_SEARCH_URL = "https://api.stackexchange.com/2.3/search/excerpts"
QUESTION_ANSWERS_URL = "https://api.stackexchange.com/2.3/questions/{id}/answers"
QUESTION_DETAIL_URL = "https://api.stackexchange.com/2.3/questions/{id}"

def make_request(url, params):
    """Helper function to make API requests with retries and error handling."""
    for _ in range(3):  # Retry up to 3 times in case of failure
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded. Waiting before retrying...")
                time.sleep(5)
            else:
                print(f"Error: Received status code {response.status_code}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(2)
    return None

with open(OUTPUT_FILE, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["question_id", "question_title", "question_body", "num_answers", "answers_with_upvotes", "majority"])

    print("Searching for answers that explicitly contain 'IANAL'...")
    page = 1
    while True:
        params = {
            'q': 'IANAL',
            'site': SITE,
            'pagesize': 100,
            'page': page,
            'filter': 'withbody',
            'item_type': 'answer',
            'key': API_KEY
        }
        data = make_request(ANSWER_SEARCH_URL, params)
        if not data or 'items' not in data:
            break

        question_ids = set()
        for item in data.get('items', []):
            if item.get('item_type') == 'answer':
                question_ids.add(item.get('question_id'))

        for question_id in question_ids:
            # Fetch question details
            quest_params = {'site': SITE, 'filter': 'withbody', 'key': API_KEY}
            quest_data = make_request(QUESTION_DETAIL_URL.format(id=question_id), quest_params)

            # Fetch all answers for the question
            ans_params = {'site': SITE, 'pagesize': 100, 'filter': 'withbody', 'key': API_KEY}
            ans_data = make_request(QUESTION_ANSWERS_URL.format(id=question_id), ans_params)

            if not quest_data or 'items' not in quest_data or not ans_data or 'items' not in ans_data:
                continue

            question_detail = quest_data['items'][0]
            question_title = question_detail.get('title', '')
            question_body = question_detail.get('body', '')
            num_answers = len(ans_data['items'])

            # Store all answers and their upvotes in a structured format
            answers_with_upvotes = []
            max_upvotes = 0
            ianal_answer_max = False
            ianal_present = False

            for answer in ans_data['items']:
                answer_id = answer.get('answer_id')
                answer_body = answer.get('body', '').replace('\n', ' ').replace('\r', ' ')
                upvotes = answer.get('score', 0)
                answers_with_upvotes.append(f"[Answer ID: {answer_id}, Upvotes: {upvotes}] {answer_body}")

                # Determine max upvoted answer
                if upvotes > max_upvotes:
                    max_upvotes = upvotes
                    ianal_answer_max = "ianal" in answer_body.lower()

                if "ianal" in answer_body.lower():
                    ianal_present = True

            # Determine majority field
            if num_answers == 1:
                majority = "NA"
            else:
                majority = "True" if ianal_present and ianal_answer_max else "False"

            writer.writerow([question_id, question_title, question_body, num_answers, " | ".join(answers_with_upvotes), majority])

            time.sleep(0.2)

        if not data.get('has_more'):
            break

        page += 1
        time.sleep(1)

print(f"Data saved to {OUTPUT_FILE}")


Searching for answers that explicitly contain 'IANAL'...
Error: Received status code 403
Data saved to uvoss.csv


In [ ]:
import pandas as pd

# Load the dataset (Replace with your actual file path if running locally)
file_path = "ianal_posts_oss.csv"
df = pd.read_csv(file_path)

# Check if the column exists
if "majority" in df.columns:  # Adjust column name if needed
    # Count occurrences of True, False, and NA values
    true_count = (df["majority"] == True).sum()
    false_count = (df["majority"] == False).sum()
    na_count = df["majority"].isna().sum()  # If NA values exist

    # Print the results
    print(f"True Count: {true_count}")
    print(f"False Count: {false_count}")
    print(f"NA Count: {na_count}")

else:
    print("Column 'upvote_comparison' not found in dataset. Please check the column names.")


True Count: 42
False Count: 52
NA Count: 131


In [ ]:
import requests
import csv
import time

"""
Script used to find entries of Stack Exchange sites that contain the acronym "IANAL" and
retrieve all answers for those questions, along with upvotes and number of answers per question.
"""

# Configuration
SITE = 'serverfault'  # Change to another Stack Exchange site as needed
OUTPUT_FILE = 'ianal_posts_server.csv'
API_KEY = 'rl_HzTM85avzXjeFbbNQArcyrx67'  # Add your API key here for higher rate limits (optional)

# API endpoints
ANSWER_SEARCH_URL = "https://api.stackexchange.com/2.3/search/excerpts"
QUESTION_ANSWERS_URL = "https://api.stackexchange.com/2.3/questions/{id}/answers"
QUESTION_DETAIL_URL = "https://api.stackexchange.com/2.3/questions/{id}"

def make_request(url, params):
    """Helper function to make API requests with retries and error handling."""
    for _ in range(3):  # Retry up to 3 times in case of failure
        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded. Waiting before retrying...")
                time.sleep(5)
            else:
                print(f"Error: Received status code {response.status_code}")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(2)
    return None

with open(OUTPUT_FILE, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["question_id", "question_title", "question_body", "num_answers", "answers_with_upvotes", "majority"])

    print("Searching for answers that explicitly contain 'IANAL'...")
    page = 1
    while True:
        params = {
            'q': 'IANAL',
            'site': SITE,
            'pagesize': 100,
            'page': page,
            'filter': 'withbody',
            'item_type': 'answer',
            'key': API_KEY
        }
        data = make_request(ANSWER_SEARCH_URL, params)
        if not data or 'items' not in data:
            break

        question_ids = set()
        for item in data.get('items', []):
            if item.get('item_type') == 'answer':
                question_ids.add(item.get('question_id'))

        for question_id in question_ids:
            # Fetch question details
            quest_params = {'site': SITE, 'filter': 'withbody', 'key': API_KEY}
            quest_data = make_request(QUESTION_DETAIL_URL.format(id=question_id), quest_params)

            # Fetch all answers for the question
            ans_params = {'site': SITE, 'pagesize': 100, 'filter': 'withbody', 'key': API_KEY}
            ans_data = make_request(QUESTION_ANSWERS_URL.format(id=question_id), ans_params)

            if not quest_data or 'items' not in quest_data or not ans_data or 'items' not in ans_data:
                continue

            question_detail = quest_data['items'][0]
            question_title = question_detail.get('title', '')
            question_body = question_detail.get('body', '')
            num_answers = len(ans_data['items'])

            # Store all answers and their upvotes in a structured format
            answers_with_upvotes = []
            max_upvotes = 0
            ianal_answer_max = False
            ianal_present = False

            for answer in ans_data['items']:
                answer_id = answer.get('answer_id')
                answer_body = answer.get('body', '').replace('\n', ' ').replace('\r', ' ')
                upvotes = answer.get('score', 0)
                answers_with_upvotes.append(f"[Answer ID: {answer_id}, Upvotes: {upvotes}] {answer_body}")

                # Determine max upvoted answer
                if upvotes > max_upvotes:
                    max_upvotes = upvotes
                    ianal_answer_max = "ianal" in answer_body.lower()

                if "ianal" in answer_body.lower():
                    ianal_present = True

            # Determine majority field
            if num_answers == 1:
                majority = "NA"
            else:
                majority = "True" if ianal_present and ianal_answer_max else "False"

            writer.writerow([question_id, question_title, question_body, num_answers, " | ".join(answers_with_upvotes), majority])

            time.sleep(0.2)

        if not data.get('has_more'):
            break

        page += 1
        time.sleep(1)

print(f"Data saved to {OUTPUT_FILE}")


Searching for answers that explicitly contain 'IANAL'...
Data saved to ianal_posts_server.csv
